<h1 style="text-align: center;">Tim "Poslednji ispit" - University of Kragujevac</h1>

<p style="text-align: center;">
  <img src="logopmfy.png" alt="University of Kragujevac Logo" width="200" height="200">
</p>

<h2 style="text-align: center;">Team Members</h2>

<ul style="list-style-type: none; text-align: center;">
  <li><strong>Vuk Lazović</strong></li>
  <li><strong>Sara Verlimirović</strong></li>
  <li><strong>Mihajlo Janković</strong></li>
</ul>


# Predstavljanje problema

.....

# Preuzimanje dataseta

In [ ]:
pip install -r requirements.txt

In [1]:
import requests,os
from tqdm import tqdm


def check_dataset():
    files = os.listdir('.')
    flag=1
    for file in files:
        if("vehicles.csv"==file):
            flag=0
    return flag

def download():
    print("Downloading dataset.....")
    url = "https://www.dropbox.com/scl/fi/hiod02ra6fa1d5f5q7bmd/vehicles.csv?rlkey=ein4k3paqkw0ashh8njtyg6ed&st=gvyd3ohd&dl=1"
    response = requests.get(url, stream=True)
    
    # Get the total file size
    total_size = int(response.headers.get('content-length', 0))
    
    with open("vehicles.csv", mode="wb") as file:
        for chunk in tqdm(response.iter_content(chunk_size=10 * 1024), total=total_size//(10*1024), unit='KB'):
            file.write(chunk)

    response = requests.get(url, stream=True)


if(check_dataset()):
    download()
else:
    print("Dataset already downloaded")

Dataset already downloaded


In [2]:
import pandas as pd
df=pd.read_csv("vehicles.csv")
df


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,...,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [6]:
list(df.columns)

['id',
 'url',
 'region',
 'region_url',
 'price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'VIN',
 'drive',
 'size',
 'type',
 'paint_color',
 'image_url',
 'description',
 'county',
 'state',
 'lat',
 'long',
 'posting_date']

In [ ]:
df.info()

In [ ]:
print(df.describe())

# Analiza

# Selekcija (vuk)

# Modeli mašinsko učenja

## Resampling (sara)

## F-regression (sara)

## Decision Tree (mixi)

## Random forest (mixi)

## Neuronske mreže (vuk)

# Zaključak (sara)

# Literatura (sara)

 ## https://chatgpt.com/

In [ ]:
pip freeze > requirements.txt